In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TextClassificationPipeline
import torch
def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def preprocess_data(data):
    input_sentences = []
    labels = []
    label_map = {}
    label_index = 0
    
    for intent in data['intents']:
        if intent['tag'] not in label_map:
            label_map[intent['tag']] = label_index
            label_index += 1

        for pattern in intent['patterns']:
            input_sentences.append(pattern)
            labels.append(label_map[intent['tag']])
    
    return input_sentences, labels, label_map

file_path = 'intents.json'
data = load_data(file_path)
input_sentences, labels, label_map = preprocess_data(data)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenized_inputs = tokenizer(input_sentences, padding=True, truncation=True, return_tensors="pt")

In [ ]:
from torch.utils.data import Dataset, DataLoader

class IntentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = IntentDataset(encodings=tokenized_inputs, labels=labels)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_map)).to(device)

optim = torch.optim.Adam(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in dataloader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_indices = torch.argmax(logits, dim=1)
            
            total += labels.size(0)
            correct += (predicted_indices == labels).sum().item()
    
    return correct / total

In [ ]:
accuracy = evaluate(model, dataloader)
print("Accuracy on the training dataset:", accuracy)

<ipython-input-8-3f5533069f81>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


KeyboardInterrupt: ignored

In [ ]:
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader

# Load the testing data from the JSON file
def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

test_data = load_data('intents_testing.json')

# Preprocess the testing data
def preprocess_data(data, label_map):
    input_sentences = []
    labels = []
    
    for intent in data['intents']:
        for pattern in intent['patterns']:
            input_sentences.append(pattern)
            labels.append(label_map[intent['tag']])
    
    return input_sentences, labels

test_inputs, test_labels = preprocess_data(test_data, label_map)

# Tokenize the testing input sentences using the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
test_encodings = tokenizer(test_inputs, truncation=True, padding=True)

# Create a IntentDataset object for the testing data
class IntentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = IntentDataset(encodings=test_encodings, labels=test_labels)

# Create a DataLoader object for the testing dataset
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Evaluate the model on the testing dataset
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_indices = torch.argmax(logits, dim=1)
            
            total += labels.size(0)
            correct += (predicted_indices == labels).sum().item()
    
    return correct / total

test_accuracy = evaluate(model, test_dataloader)
print("Accuracy on the testing dataset:", test_accuracy)


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# Evaluate the model on a dataset
def evaluate(model, dataloader):
    model.eval()
    predicted_labels = []
    true_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_indices = torch.argmax(logits, dim=1)
            
            predicted_labels += predicted_indices.tolist()
            true_labels += labels.tolist()
    
    # Compute precision, recall, and f1 score
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
    
    return precision, recall, f1

# Evaluate the model on the training dataset
train_precision, train_recall, train_f1 = evaluate(model, dataloader)
print('Training precision: {:.4f}'.format(train_precision))
print('Training recall: {:.4f}'.format(train_recall))
print('Training f1 score: {:.4f}'.format(train_f1))

# Evaluate the model on the testing dataset
test_precision, test_recall, test_f1 = evaluate(model, test_dataloader)
print('Testing precision: {:.4f}'.format(test_precision))
print('Testing recall: {:.4f}'.format(test_recall))
print('Testing f1 score: {:.4f}'.format(test_f1))


In [ ]:
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold

# Load the data from the JSON file
def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

data = load_data('intents.json')

# Preprocess the data
def preprocess_data(data):
    input_sentences = []
    labels = []
    label_map = {}
    label_index = 0

    for intent in data['intents']:
        if intent['tag'] not in label_map:
            label_map[intent['tag']] = label_index
            label_index += 1

        for pattern in intent['patterns']:
            input_sentences.append(pattern)
            labels.append(label_map[intent['tag']])

    return input_sentences, labels, label_map

input_sentences, labels, label_map = preprocess_data(data)

# Tokenize the input sentences using the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_inputs = tokenizer(input_sentences, truncation=True, padding=True)

# Create a IntentDataset object for the data
class IntentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = IntentDataset(encodings=tokenized_inputs, labels=labels)

# Set up the cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

# Iterate over the folds
for train_idx, val_idx in kfold.split(dataset):
    print(f"Fold {fold}")
    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    val_dataset = torch.utils.data.Subset(dataset, val_idx)
    train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    # Create a BERT model for sequence classification
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map)).to(device)

    # Train the model on the training dataset
    optim = torch.optim.Adam(model.parameters(), lr=5e-5)

    for epoch in range(3):
        for batch in train_dataloader:
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optim.step()

    # Evaluate the model on the validation dataset
    val_accuracy = evaluate(model, val_dataloader)
    print("Accuracy on the validation dataset:", val_accuracy)

    fold += 1

# Evaluate the model on the testing dataset
test_data = load_data('intents_testing.json')
test_inputs, test_labels = preprocess_data(test_data, label_map)
test_encodings = tokenizer(test_inputs, truncation=True, padding=True)
test_dataset = IntentDataset(encodings=test_encodings, labels=test_labels)


Fold 1


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Accuracy on the validation dataset: 0.3142857142857143
Fold 2


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Accuracy on the validation dataset: 0.4228571428571429
Fold 3


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Accuracy on the validation dataset: 0.08620689655172414
Fold 5


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Accuracy on the validation dataset: 0.367816091954023


TypeError: ignored